In [ ]:
import pandas as pd
import numpy as np
import itertools
import collections

In [ ]:
%pylab inline
figsize(15,10)

For each of our three scenarios we created a randomized set of queries on a list of 5 objects, normalized to 1. The distribution of each was proportioned such that it modeled the characteristics of of each senario as shown below.

## "Big Event" Scenario

The "big event" is a scenario where some large news event ocurres skewing the distribution of queries suddenly and persiting for some time. We did this by first having an unbiased random distribution of queries for some time then skewing them in favor of one particulat query, as shown by the distribution graph below. This kind of scenario is useful for a self organizing list because the "big event" can be pulled to the frount of the list making it more eficient to find results on when it is a common search.

In [ ]:
sample_data1 = pd.DataFrame(np.abs(np.random.randn(50,5)))
sample_data1 = sample_data1.apply(lambda x: x/x.sum(), axis=1)

In [ ]:
sample_data2 = pd.DataFrame(np.abs(np.random.randn(50,5)),index=range(50,100))
sample_data2.ix[:,0] += 25
sample_data2 = sample_data2.apply(lambda x: x/x.sum(), axis=1)

In [ ]:
big_event_dist = pd.concat([sample_data1,sample_data2]).cumsum(axis=1)
big_event_queries = np.random.ranf(100)
big_event_dist['rand'] = big_event_queries

### Queries for "Big Event"

In [ ]:
big_event_queries = 4-big_event_dist.apply(lambda x: (x.iloc[:4] > x.rand).sum(),axis=1)
big_event_queries

### Distribution of "Big Event" queries

In [ ]:
pd.concat([sample_data1,sample_data2]).plot(kind='area')
title('Big Event')
_ = legend(framealpha=1,loc='best')

## "Viral Video" Scenario

Viral videos do not exist one day and then suddenly rise to poularity before people lose intrest and the video fades into obscurity. The simulation is fairly similar to the big event case only that the querrie does not exist before its rise and relativly short period of time it fades into the backround as average as any other query. This give a simulation of a search falling out of high priority and can be useful to show how these algorithms sort themselves out on a downtern in searches.

In [ ]:
sample_data3 = pd.DataFrame(np.abs(np.random.randn(50,5)),index=range(50,100))
sine = np.sin((sample_data3.index.values+0.0)/sample_data3.index.values.size*2*np.pi)
sine[sine < 0] = 0
sample_data3.ix[:,0] += sine*10
sample_data3.ix[sample_data3.ix[:,0] < 0,0] = 0
sample_data3 = sample_data3.apply(lambda x: x/x.sum(), axis=1)

In [ ]:
sample_data4 = pd.DataFrame(np.abs(np.random.randn(50,5)))
sample_data4.ix[:,0] = 0
sample_data4 = sample_data4.apply(lambda x: x/x.sum(), axis=1)

In [ ]:
viral_dist = pd.concat([sample_data4,sample_data3]).cumsum(axis=1)
viral_queries = np.random.ranf(100)
viral_dist['rand'] = viral_queries

### Queries for "Viral Video"

In [ ]:
viral_queries = 4-viral_dist.apply(lambda x: (x.iloc[:4] > x.rand).sum(),axis=1)
viral_queries

### Distribution of "Viral Video" queries

In [ ]:
pd.concat([sample_data4,sample_data3]).plot(kind='area')
_ = legend(framealpha=1,loc='best')

## Lopsided but constant distribtion

For this last scenario we wanted to see how each algorithm would work in a very steady state, with each query being uniformly distributed except for one which it is biased towards. No real world case would be this steady but we felt it was important to have a base case for comparison. 

### "Lopsided but constant distribution" queries

In [ ]:
constant_dist = pd.DataFrame(np.ones((100,5)),index=range(100))
constant_dist.ix[:] = 1./3/4
constant_dist.ix[:,0] = 2./3
constant_dist['rand'] = np.random.ranf(100)
constant_queries = 4-constant_dist.cumsum(axis=1).apply(lambda x: (x.iloc[:4] > x.rand-1).sum(),axis=1)
constant_queries

### "Lopsided but constant distribution" distribution

In [ ]:
constant_dist.iloc[:,:5].plot(kind='area')
_ = legend(loc='best', framealpha=1)

# Implementations

## Regular Linked list

In [ ]:
class LinkedList:
    def __init__(self):
        self.head = None
        
    def push(self,item):
        self.head = [item,self.head]
        
    def pop(self):
        popped,self.head = self.head
        return popped
    
    def insert(self,item,index):
        if index == 0:
            self.head = (item,self.head)
        else:
            parent_node = self.get(index-1)
            parent_node[1] = [item,parent_node[1]]
            
                
    def get(self,index):
        node = self.head
        for i in xrange(index):
            _,node = node
        return node
    
    def _search(self, item):
        node= self.head
        index = 0
        while node is not None:
            if node[0] == item:
                return node,index
            _,node = node
            index += 1
        return None,None
    
    def search_index(self, item):
        node,index = self._search(item)
        
    def search_node(self, item):
        node,index = self._search(item)
    
    def remove(self, index):
        if index == 0:
            _, self.head = self.head
        else:
            parent_node = self.get(index-1)
            parent_node[1] = parent_node[1][1]
            
    def append(self,item):
        if self.head == None:
            self.push(item)
        else:
            node = self.head
            while node[1] is not None:
                _,node = node
            node[1] = [item,None]
    
    def __repr__(self):
        node = self.head
        items = []
        while node is not None:
            item,node = node
            items.append(str(item))
        return ' -> '.join(items)

#### Test it out

In [ ]:
a = LinkedList()
a.append(1)
a.append(2)
a.append(3)
a.get(0)
a.insert(5,2)
a.remove(2)
a

## Regular Linked List Simulations

In [ ]:
def create_list():
    test_list = LinkedList()
    test_list.append(4)
    test_list.append(3)
    test_list.append(2)
    test_list.append(1)
    test_list.append(0)
    return test_list

def test_list_with_big_event():
    test_list = create_list()
    for query in big_event_queries:
        test_list.search_node(query)
        
def test_list_with_viral():
    test_list = create_list()
    for query in viral_queries:
        test_list.search_node(query)
        
def test_list_with_constant():
    test_list = create_list()
    for query in constant_queries:
        test_list.search_node(query)

### Time to create the list

In [ ]:
%timeit create_list()

### Time to field the "Big Event" queries

In [ ]:
%timeit test_list_with_big_event()

### Time to field "Viral Video" queries

In [ ]:
%timeit test_list_with_viral()

### Time to field "Lopsided but constant distribution" queries

In [ ]:
%timeit test_list_with_constant()

# Part 2

# Move to Front
This technique moves the element which is accessed to the head of the list.

In [ ]:
class MoveToFrontList(LinkedList):
    def move_to_front(self,item):
        node = self.head
        if node[0] == item:
            return node[0]
        while node is not None:
            if node[0] == item:
                parent[1] = node[1]
                self.push(node[0])
                return node[0]
            parent = node
            node = node[1]
            

#### Test it out

In [ ]:
a = MoveToFrontList()
a.append(1)
a.append(2)
a.append(3)
a

In [ ]:
a.move_to_front(3)
a

In [ ]:
a.move_to_front(2)
a

In [ ]:
a.move_to_front(3)
a

In [ ]:
test_mtf_list_5 = MoveToFrontList()
test_mtf_list_5.append(4)
test_mtf_list_5.append(3)
test_mtf_list_5.append(2)
test_mtf_list_5.append(1)
test_mtf_list_5.append(0)
test_mtf_list_5

## Move-to-Front Simulations

In [ ]:
def create_list():
    test_mtf_list = MoveToFrontList()
    test_mtf_list.append(4)
    test_mtf_list.append(3)
    test_mtf_list.append(2)
    test_mtf_list.append(1)
    test_mtf_list.append(0)
    return test_mtf_list

def test_move_to_front_with_big_event():
    test_mtf_list = create_list()
    for query in big_event_queries:
        test_mtf_list.move_to_front(query)
        
def test_move_to_front_with_viral():
    test_mtf_list = create_list()
    for query in viral_queries:
        test_mtf_list.move_to_front(query)
        
def test_move_to_front_with_constant():
    test_mtf_list = create_list()
    for query in constant_queries:
        test_mtf_list.move_to_front(query)

### Time to create the list

In [ ]:
%timeit create_list()

### Time to field the "Big Event" queries

In [ ]:
%timeit test_move_to_front_with_big_event()

### Time to field "Viral Video" queries

In [ ]:
%timeit test_move_to_front_with_viral()

### Time to field "Lopsided but constant distribution" queries

In [ ]:
%timeit test_move_to_front_with_constant()

# Count Method
In this technique, the number of times each node was searched for is counted i.e. every node keeps a separate counter variable which is incremented every time it is called. The nodes are then rearranged according to decreasing count.

In [ ]:
class CountList(LinkedList):
        
    def search(self, item):
        node= self.head
        index = 0
        while node is not None:
            if node[0][0] == item:
                node[0][1] += 1
                self.remove(index)
                self.add(node[0])
                return 
            _,node = node
            index += 1
        
    def add(self, new_node):
        node = self.head
        if node is None or new_node[1] > node[0][1]:
            self.head = [new_node,node]
            return
        while node[1] is not None:
            parent = node
            _,node = node   
            if new_node[1] > node[0][1]:
                parent[1] = [new_node,parent[1]]
                #print 'Adding {} after {}'.format(str(new_node),str(parent[0]))
                return
        #print 'Adding {} at end'.format(str(new_node))
        node[1] = [new_node,None]

#### Test it out

In [ ]:
a = CountList()
a.add([1,0])
a.add([2,0])
a.add([3,0])
print 'Start:         ',a
a.search(3)
print 'Search 3:      ',a
a.search(3)
print 'Search 3 again:',a
a.search(2)
print 'Search 2:      ',a
a.search(2)
print 'Search 2 again:',a
a.search(2)
print 'Search 2 again:',a

## Count Method Simulations

In [ ]:
def create_list():
    test_count_list = CountList()
    test_count_list.add([4,0])
    test_count_list.add([3,0])
    test_count_list.add([2,0])
    test_count_list.add([1,0])
    test_count_list.add([0,0])
    return test_count_list

def test_count_with_big_event():
    test_count_list = create_list()
    for query in big_event_queries:
        test_count_list.search(query)
        
def test_count_with_viral():
    test_count_list = create_list()
    for query in viral_queries:
        test_count_list.search(query)
    
def test_count_with_constant():
    test_count_list = create_list()
    for query in constant_queries:
        test_count_list.search(query)

### Time to create the list

In [ ]:
%timeit create_list()

### Time to field the "Big Event" queries

In [ ]:
%timeit test_count_with_big_event()

### Time to field "Viral Video" queries

In [ ]:
%timeit test_count_with_viral()

### Time to field "Lopsided but constant distribution" queries

In [ ]:
%timeit test_count_with_constant()

#Transpose Method
This technique involves swapping an accessed node with its predecessor. Therefore, if any node is accessed, it is swapped with the node in front unless it is the head node, thereby increasing its priority. 

In [ ]:
class TransposeList(LinkedList):
    def search(self,item):
        node = self.head
        if node[0] == item:
            return node
        while node is not None:
            if node[0] == item:
                parent[0],node[0] = node[0],parent[0]
                return parent
            parent = node
            _,node = node             

#### Test it out

In [ ]:
a = TransposeList()
a.append(1)
a.append(2)
a.append(3)
print 'Start:   ',a
a.search(3)
print 'Search 3:',a
a.search(3)
print 'Search 3:',a
a.search(1)
print 'Search 1:',a

## Transpose Simulations

In [ ]:
def create_list():
    test_transpose_list = CountList()
    test_transpose_list.add([4,0])
    test_transpose_list.add([3,0])
    test_transpose_list.add([2,0])
    test_transpose_list.add([1,0])
    test_transpose_list.add([0,0])
    return test_transpose_list

def test_transpose_with_big_event():
    test_transpose_list = create_list()
    for query in big_event_queries:
        test_transpose_list.search(query)
    

def test_transpose_with_viral():
    test_transpose_list = create_list()
    for query in viral_queries:
        test_transpose_list.search(query)
        

def test_transpose_with_constant():
    test_transpose_list = create_list()
    for query in constant_queries:
        test_transpose_list.search(query)

### Time to create the list

In [ ]:
%timeit create_list()

### Time to field the "Big Event" queries

In [ ]:
%timeit test_transpose_with_big_event()

### Time to field "Viral Video" queries

In [ ]:
%timeit test_transpose_with_viral()

### Time to field "Lopsided but constant distribution" queries

In [ ]:
%timeit test_transpose_with_constant()